In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [3]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

/content/gdrive/My Drive/Kaggle


In [4]:
#!kaggle datasets download -d aashita/nyt-comments

In [5]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import string, os
import pandas as pd
import numpy as np
from keras import utils as np_utils

In [6]:
import pandas as pd
import string
songs = pd.read_csv('/content/gdrive/My Drive/Kaggle/franz_ferdinand.csv')
lyrics = songs.lyrics
lyrics_cnt = len(lyrics)

all_lyrics = []
all_lyrics.extend(list(lyrics.values))
#print(all_lyrics)

for a in all_lyrics:
  if not isinstance(a, str):
    all_lyrics.remove(a)

print(len(all_lyrics))
all_lyrics = all_lyrics[:200]

translator = str.maketrans('', '', string.punctuation)

# poetry = pd.read_csv('/content/gdrive/My Drive/Kaggle/PoetryFoundationData.csv',quotechar='"')

# poetry.head()

# poem = poetry.Poem
# poem_cnt = len(poem)

# all_poems = []
# all_poems.extend(list(poem.values))


# for a in all_poems:
#   if not isinstance(a, str):
#     all_poems.remove(a)

# all_poems = all_poems[:50]
# joined = all_poems+all_lyrics







470


In [7]:

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        #print(line)
        token_list = tokenizer.texts_to_sequences([line])[0]
        #print(token_list)

        for i in range(1, len(token_list)):
                n_gram_sequence = token_list[:i+1]
                input_sequences.append(n_gram_sequence)
        #print(input_sequences)

    return input_sequences, total_words



def get_padded_sequences(input_sequences):
    max_seq_len = max( [len(x) for x in input_sequences])
    padded_input_sequences =  np.array(pad_sequences(input_sequences, 
                                                     maxlen=max_seq_len, padding='pre'))
    return padded_input_sequences, max_seq_len



def prep_train_data(padded_input_sequences,total_words):
    
    x_train = padded_input_sequences[:,:-1]
    labels = padded_input_sequences[:,-1]
    labels = keras.utils.np_utils.to_categorical(labels, num_classes=total_words)
    
    return x_train, labels



In [8]:

corpus = [clean_text(x) for x in all_lyrics]
tokenizer = Tokenizer()
input_sequences, total_words = get_sequence_of_tokens(corpus)
padded_input_sequences, max_seq_len = get_padded_sequences(input_sequences)
x_train, labels = prep_train_data(padded_input_sequences,total_words)

In [9]:
def lstm_model(max_seq_len,total_words):
    
    input_len = max_seq_len - 1 #zadnju predvidam
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(128))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = lstm_model(max_seq_len,total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 526, 10)           21910     
                                                                 
 lstm (LSTM)                 (None, 128)               71168     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2191)              282639    
                                                                 
Total params: 375,717
Trainable params: 375,717
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(x_train, labels, epochs=60, verbose=5)

Epoch 1/60


KeyboardInterrupt: ignored

In [ ]:
def generate_lyrics(seed_txt, next_words_cnt , max_seq_len, model):
  
    for i in range(0, next_words_cnt):
        token_list = tokenizer.texts_to_sequences([seed_txt])[0]
        padded_token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        prediction = np.argmax(model.predict(padded_token_list), axis=-1)

        #print('prediction',prediction)
        #tokenizer.word_index.items() --> rijecnik (rijec,index)
        
        
        for (word, index) in tokenizer.word_index.items():
            output_word = ""
            if (prediction == index):
                seed_txt += " " + word
                break
      
        
    return seed_txt.title()

In [ ]:
#model.save("model_franzi")

In [ ]:
import random
unique = []
for i in range(0,len(corpus)):
    unique.append(corpus[i].split())


for i in range(0,20):
  random_list = unique[random.randint(0,len(unique)-1)]
  seed = random_list[random.randint(0,len(random_list))]
  print( generate_lyrics(seed,random.randint(3,15),max_seq_len, model ) )






